In [4]:
import json
from pathlib import Path

# --- Configuração ---
# Altere este nome se o seu diretório principal for outro
ROOT_DIRECTORY_NAME = 'stock_TRANSFORMER_tuning_weighted'
# --------------------

def rank_keras_tuner_trials(root_dir_name):
    """
    Varre um diretório do KerasTuner, lê todos os arquivos 'trial.json',
    extrai o ID e o score, e imprime um ranking ordenado.
    """
    
    root_dir = Path(root_dir_name)
    
    if not root_dir.is_dir():
        print(f"Erro: Diretório '{root_dir_name}' não encontrado.")
        print("Certifique-se de que o script está no lugar certo.")
        return

    results = []

    # 1. Encontrar e processar todos os arquivos 'trial.json'
    # 'rglob' faz uma busca recursiva (entra em todas as subpastas)
    print(f"Procurando por trials em '{root_dir_name}'...")
    
    for trial_file in root_dir.rglob('trial.json'):
        try:
            with open(trial_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # Extrai os dados necessários do JSON
                trial_id = data.get('trial_id')
                score = data.get('score')
                status = data.get('status')

                # Adiciona na lista apenas se o trial foi completado e tem score
                if status == 'COMPLETED' and trial_id is not None and score is not None:
                    results.append({
                        'id': trial_id,
                        'score': score
                    })
                elif status != 'COMPLETED':
                    print(f"  > Pulando trial {trial_id} (Status: {status})")

        except json.JSONDecodeError:
            print(f"Erro: Falha ao ler o JSON em {trial_file}")
        except Exception as e:
            print(f"Erro inesperado ao processar {trial_file}: {e}")

    # 2. Ordenar (Rankear) os resultados
    if not results:
        print("Nenhum trial 'COMPLETED' com score foi encontrado.")
        return

    # Ordena a lista 'results' com base no valor da chave 'score'
    # 'reverse=True' garante que será do maior para o menor
    sorted_results = sorted(results, key=lambda item: item['score'], reverse=True)

    # 3. Exibir o Ranking
    print("\n--- Ranking Final dos Trials (Maior Score para Menor) ---")
    print("---------------------------------------------------------")
    for rank, trial in enumerate(sorted_results, 1):
        # ':02d' formata o rank (1 -> 01), ':.6f' formata o score
        print(f"Rank {rank:02d}: Trial {trial['id']} \t- Score: {trial['score']:.12f}")
    print("---------------------------------------------------------")


# --- Executa a função ---
if __name__ == "__main__":
    rank_keras_tuner_trials(ROOT_DIRECTORY_NAME)

Procurando por trials em 'stock_TRANSFORMER_tuning_weighted'...
  > Pulando trial 0003 (Status: RUNNING)

--- Ranking Final dos Trials (Maior Score para Menor) ---
---------------------------------------------------------
Rank 01: Trial 0001 	- Score: 0.587478935719
Rank 02: Trial 0002 	- Score: 0.584629893303
Rank 03: Trial 0000 	- Score: 0.583118617535
---------------------------------------------------------
